# Function Calling & Tool Use - Giving LLMs Superpowers

Hello everyone. Today, we explore a feature that elevates an LLM from a simple text generator into an active agent that can interact with the outside world: **Function Calling**, also known as **Tool Use**.

Until now, the LLM has been in a closed box. It can't browse the web, access your database, or call an API. Function calling is the mechanism that breaks open that box. It allows the model to ask *our application* to run a function and return the result, giving it access to live data and real-world capabilities.

This notebook will walk you through a complete, end-to-end example of building a simple weather bot that uses a "tool" (a local Python function) to get real-time information.

## Defining Our "Tool"

First, let's define the tool we want to give to the LLM. For this demonstration, we'll create a mock weather function. In a real application, this function would call a live weather API.

Our tool is a simple Python function called `get_current_weather`.

In [ ]:
import litellm
import json
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    return litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

# --- Our Local Tool (Python Function) ---
def get_current_weather(location: str) -> str:
    """
    Gets the current weather for a given location.

    Args:
        location (str): The city and state, e.g., "San Francisco, CA".

    Returns:
        A JSON string with the weather information.
    """
    print(f"--- TOOL: Calling get_current_weather(location='{location}') ---")
    # In a real app, this would call a live weather API.
    # We'll use mock data for this example.
    if "boston" in location.lower():
        return json.dumps({"location": "Boston", "temperature": "18°C", "forecast": "cloudy"})
    elif "berlin" in location.lower():
        return json.dumps({"location": "Berlin", "temperature": "15°C", "forecast": "sunny"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

print("Setup complete. Helper functions and context are ready.")

In [ ]:
print(get_current_weather.__doc__)

## Describing the Tool to the Model

We don't send the model our Python code. Instead, we send a **description** of the tool. This description follows a specific JSON schema, defining the function's name, its purpose, and the parameters it accepts.

In [ ]:
tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": get_current_weather.__doc__,
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, for example, San Francisco, CA"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

print("Tool schema is defined.")

## The First API Call - Model Decides to Use the Tool

Now, we make our first API call. We pass the user's question and the `tools` schema. The model will see the question, realize it can't answer from its internal knowledge, and see that our `get_current_weather` tool is a perfect fit.

Instead of returning a text message, it will return a special `tool_calls` object.

In [ ]:
user_prompt = "What's the weather like in Berlin?"
messages = [
    {
        "role": "user",
        "content": user_prompt
    }
]

first_response = get_completion(
    messages,
    tools=tools_schema
)

In [ ]:
response_message = first_response.choices[0].message
print(response_message)

if response_message.tool_calls:
    print("\nConclusion: the model wants to call a function!")
else:
    print("\nConclusion: the model doesn't want to call a function!")

## Executing the Tool and Sending the Result Back

Our code now needs to handle this "tool call" request. We will:

1. Parse the `tool_calls` object to get the function name and arguments.
2. Call our local Python function with those arguments.
3. Append the result of our function call to the message history with a new `tool` role.
4. Make a second API call with this updated history.

In [ ]:
tool_call = response_message.tool_calls[0]
function_name = tool_call.function.name
function_args = json.loads(tool_call.function.arguments)
messages_copy = messages.copy()

if function_name == "get_current_weather":
    tool_res = get_current_weather(**function_args)
    print(f"Result from our tool: {tool_res}")

    messages_copy.append(response_message)
    messages_copy.append({
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": tool_res
    })

    print("\n--- Making API request with tool results")

    final_response = get_completion(messages_copy)
    
    print(f"\nFinal human-readable response: {final_response.choices[0].message.content}")